In [4]:
import itertools as it
import copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import statistics

#Remove cubes from consideratiom that share no corners in common with the target
#Input: dictionary_of_cubes (C32, C33, C34, C35, C36)
#       target_cube a list of eight corner numbers

def CollectionsToMakeTarget(dictionary_of_cubes, target_cube):
    #make a list of cubes that share at least one corner type with target_cube
    listtomake=[]
    for key in dictionary_of_cubes:
        if set(target_cube) & set(dictionary_of_cubes[key]):
            listtomake.append(dictionary_of_cubes[key])
        else:
            print(target_cube, "and", dictionary_of_cubes[key], "no intersection")
        
    Collections=list(it.combinations(listtomake, 8))
    print("There are", len(Collections), "collections to make the target", target_cube)
    
    return Collections


#make a list of cubes that share at least one corner type with the target
#and create collections of 8 cubes, all of which contain the included cubes.
    
def RestrictedCollectionsToMakeTarget(dictionary_of_cubes, target_cube, tuple_cube_names):
    
    #create a list of cubes to include in all collections -- append later
    includelist=list(dictionary_of_cubes[x] for x in tuple_cube_names)
    
    #the number of cubes that will be appended
    number = len(includelist)
    listtomake=[]
    local_dict = copy.deepcopy(dictionary_of_cubes)
    
    for x in tuple_cube_names:
        local_dict.pop(x)

    for key in local_dict:
        if set(target_cube) & set(dictionary_of_cubes[key]):
            listtomake.append(dictionary_of_cubes[key])
        else:
            print(target_cube, "and", dictionary_of_cubes[key], "no intersection")
        
    RestCollections=list(it.combinations(listtomake, 8-number))
    
    RestrictedCollections=[]
    for x in RestCollections:
        x=list(x)
        x.extend(includelist)
        x= tuple(x)
        RestrictedCollections.append(x)
        
    print("There are", len(RestCollections), "collections to make the target", target_cube)
    
    return RestrictedCollections
    
    

def CountFrequency(my_list):
    my_list.sort(reverse=True)  
    # Creating an empty dictionary 
    freq = {}
    for items in my_list:
        freq[items] = my_list.count(items)
      
    for key, value in freq.items():
        print ("% d : % d"%(key, value))

        
def SolutionFrequencyList(dictionary_of_cubes, target_cube):
    collections_for_target=CollectionsToMakeTarget(dictionary_of_cubes, target_cube)
    solutions=[]
    zero_counter=0
    check_progress=0
    for k in collections_for_target:
        check_progress= check_progress+1
        if check_progress%10000==0:
            print(".")
        
        soln_num = CountingSolutions(CornerCountVector(k, target_cube))
        if soln_num > 0:
            solutions.append(soln_num)
        else:
            zero_counter=zero_counter+1
    
    #print("Checked", len(collections_for_target), "collections of 8 cubes")
    #print(zero_counter, "of the collections yield no solutions")
     
    return CountFrequency(solutions)

        

def RestrictedSolutionFrequencyList(dictionary_of_cubes, target_cube, tuple_cube_names):
    collections_for_target=RestrictedCollectionsToMakeTarget(dictionary_of_cubes, target_cube, tuple_cube_names)
    solutions=[]
    zero_counter=0
    check_progress=0
    for k in collections_for_target:
        check_progress= check_progress+1
        if check_progress%10000==0:
            print(".")
        
        soln_num = CountingSolutions(CornerCountVector(k, target_cube))
        if soln_num > 0:
            solutions.append(soln_num)
            #if soln_num == 16:
                #print(k)
        else:
            zero_counter=zero_counter+1
    
    print("Checked", len(collections_for_target), "collections of 8 cubes")
    print(zero_counter, "of the collections yield no solutions")
    
    return CountFrequency(solutions)



def CountFrequencyEntry(my_list, entry):
    counter=0
    for items in my_list:
        if items == entry:
            counter=counter+1
            
    return counter


#Input: Eight_Cubes, a list of eight lists containing the corner numbers of eight distinct cubes
#       Target_Cube, a list of the eight corner numbers of the target cube 
#Calculates: associate the 8 cubes with the first 8 primes (cube i has index i^th prime)
#                [cube1, cube2, ..., cube 8] <-> [2, 3, 5, 7, 11, 13, 17, 19]
#            for each corner number `xyz` of the target
#                j= the count of cubes in Eight_Cubes containing corner number xyz
#                record the prime indicies of the cubes in Eight_Cubes that contain xyz [p_i1, p_i2, ..., p_ij]
#                record the multiplicity mk = (the number of times xyz appears in cube of index p_ik)
#Output: a vector with entries for each corner number `xyz` of the target
#       [[xyz, j, [p_i1^m1, p_i2^m2, ..., p_ij^mj], .... , [441, 5, [2^3, 5^2, 7^1, 11^3, 17^2]]]
def CornerCountVector(Eight_Cubes, Target_Cube):
    listCounts=[]
    primes=[2,3,5,7,11,13,17,19]
    for x in Target_Cube:   
        count=0
        index=[]
        for k in range(0,8):
            if x in Eight_Cubes[k]:
                multiplicity=CountFrequencyEntry(Eight_Cubes[k], x)
                index.append(primes[k]**multiplicity)
                count=count+1
        listCounts.append([x, count, index]) 
        # for each corner x of the target cube, count=number of cubes in the collection that have that corner, 
        # index=[collection number that contains that corner]
    # print("list counts (TargetCorner, #CollectionCorners, CubeIndex):", listCounts)    
    return(listCounts)

#Input: corner count vector for Eight_Cubes and Target_Cube
#Output: number of distinct solutions for Eight_Cubes and Target_Cube 
#  each prime appears with some multiplicity and multiple solutions according to the multiplicity
def CountingSolutions(Counts):
    for x in Counts:
        # checking for a target corner number that no cubes in the collection can fill
        if x[1]==0:
            return 0  #No solutions possible if there is a target corner in none of the cubes
    count=0
    
    SetProducts=set(it.product(list(Counts[0][2]), list(Counts[1][2]),list(Counts[2][2]),list(Counts[3][2]),list(Counts[4][2]),list(Counts[5][2]),list(Counts[6][2]),list(Counts[7][2])))
    
    for atuple in SetProducts:
        if (atuple[0]*atuple[1]*atuple[2]*atuple[3]*atuple[4]*atuple[5]*atuple[6]*atuple[7])%(2*3*5*7*11*13*17*19)==0:
            local_count=1
            #print("yes", atuple, MultiplicityOfTuple(atuple))
            local_count=local_count*MultiplicityOfTuple(atuple)
            count=count+local_count
    
    return count

#NOTE!! The CountingSolutions code doesn't work if the target cube contains duplicate corner numbers.  
#Check composition notebook for fix ideas
#NOTE end.

#input: a tuple of numbers that contains a power of p, for p=2, 3, 5, 7, 11, 13, 17, 19
#       as one of its entries
#output: the exponent of the power of 2 in that entry

def Exponent_of_p(atuple,aprime):
    p=aprime
    for x in atuple:
        if x%p==0:
            if x%p==0 and x%(p**2)!= 0:
                exp2=1
                return exp2
            elif x%(p**2)==0 and x%(p**3)!=0:
                exp2=2
                return exp2
            elif x%(p**3)==0 and x%(p**4)!=0:
                exp2=3
                return exp2
            elif x%(p**4) ==0 and x%(p**5)!=0:
                exp2=4
                return exp2
            elif x%(p**5) ==0 and x%(p**6)!=0:
                exp2=5
                return exp2
            elif x%(p**6) ==0 and x%(p**7)!=0:
                exp2=6
                return exp2
            elif x%(p**7) ==0 and x%(p**8)!=0:
                exp2=7
                return exp2
            elif x%(p**8) ==0:
                exp2=8
                return exp2
       
    
def MultiplicityOfTuple(atuple):
    primes=[2, 3, 5, 7, 11, 13, 17, 19]
    multi=1
    for p in primes:
        multi=multi*Exponent_of_p(atuple, p)
    
    return multi

#Input: a target cube name "123.456" and a list of cube names ['123.456, ..]' from C3n
#Output: for each listed cube, the usable corner count for the target cube.
def UsableCornerCountUnordered(Target, list_of_cube_names, dictionary_of_cubes):
    UsableCornerCount=[]
    for cube in list_of_cube_names:
        cornersOfCube = dictionary_of_cubes[cube]
        count=0
        RemoveDuplicatesTarget = [*set(Target)]
        for x in RemoveDuplicatesTarget:  
            if CountFrequencyEntry(cornersOfCube, x)>0:
                count=count+CountFrequencyEntry(cornersOfCube, x)
        UsableCornerCount.append([cube, count])
    
    return UsableCornerCount

# Python code to sort the lists using the kth element of sublists
# Inplace way to sort, use of third variable
def Sort(sub_li, k):
    l = len(sub_li)
    for i in range(0, l):
        for j in range(0, l-i-1):
            if (sub_li[j][1] > sub_li[j + 1][1]):
                tempo = sub_li[j]
                sub_li[j]= sub_li[j + 1]
                sub_li[j + 1]= tempo
    return sub_li

 
def UCC(Target, dictionary_of_cubes):
    list_of_cube_names = dictionary_of_cubes.keys()
    unorder = UsableCornerCountUnordered(Target,list_of_cube_names, dictionary_of_cubes)
    ordered = list(reversed(Sort(unorder,1)))
    return ordered

def UCCsublist(ucc_output, n):
    sublist_keynames = []
    for x in ucc_output:
        if x[1] > n-1:
            sublist_keynames.append(x[0])
    return sublist_keynames
    
# I don't think this is used anywhere else.             4.9.2024
#def UCCdistribution(Target_Cube, dictionary_of_cubes):
#    list_of_names = list(dictionary_of_cubes.keys())
#    distribution=[]
#    A = len(UCCsublist(UCC(Target, list_of_names, C34), 8)) 
#    distribution.append(A)
#    for i in range(7,-1,-1):
#        B =len(UCCsublist(UCC(Target, list_of_names, C34), i))
#        C =len(UCCsublist(UCC(Target, list_of_names, C34), i+1))
#        new = B-C
#        distribution.append(new)
#    
#    return distribution


def CountIfPositive(my_list):
    count=0
    for i in range(len(my_list)):
        if my_list[i]>0:
            count=count+1
    
    return count

#Input: dictionary_of_cubes, such as C36, C35, ...
#Output: a randomly selected collection of eight cubes from the dictionary
## format of output, a list [[corner numbers cube 1], [corner numbers cube 2], ... , [corner numbers cube 8]]
def RandomEightCubes(dictionary_of_cubes): 
    #a list of all dictionary keys, length of the list
    allkeysList = list(dictionary_of_cubes.keys())
    Number=len(allkeysList)
    #a random sample of 8 numbers from a range of 0-Number
    x=random.sample(range(Number), 8)
    cube_names=[]
    collection=[]
    
    for i in x:
        cube_names.append(allkeysList[i])
    
    for j in cube_names:
        collection.append(dictionary_of_cubes[j])   
        
    return collection

#Inputs: dictionary of cubes to use for the collections of 8 cubes
#n, integer number of randomly sampled collections from dictionary 
def DistributionRandomCollectionsHowManyTargetsCanBeBuilt(dictionary_of_cubes, n):
    keysList = list(dictionary_of_cubes.keys())
    distribution_positives=[]
    
    for j in range(n):
        collection = RandomEightCubes(dictionary_of_cubes)
        a_solution_list=[]
        
        for i in keysList: 
            a_solution_list.append(CountingSolutions(CornerCountVector(collection, dictionary_of_cubes[i])))
        
        distribution_positives.append(CountIfPositive(a_solution_list))
            
    return distribution_positives


#Input: a dictionary from which all colletions of eight cubes will be considered
#.      a dictionary for the target cubes
#Output: a vector that gives the distribution of the number of buildable cubes for all collections
# NOTE, right now this code is designed for C6.  Update for new dictionaries if there are collections 
# that can be used to build more than 5 distinct cubes from the dictionary
# format of output is a vector with 7 entries 
# [#collectionsBuildNoTargets,#collectionsBuildOneTarget,#collectionsBuildTwoTargets,#collectionsBuildThreeTargets, 
#  #collectionsBuildFourTargets, #collectionsBuildFiveTargets, #collectionsBuild6+Targets]
def DistribOfNumBuildable_AllCollectionsEightCubes(dictionary_for_collections, dictionary_for_targets): 
    keysList_collections = list(dictionary_for_collections.keys())
    keysList_targets = list(dictionary_for_targets.keys())
    #make a list of all collections of 8. For C6 there are 5852925 such collections.
    #output: a list of keys
    Collections=list(it.combinations(keysList_collections, 8))
    #distribution_positives=[]
    No_targets_built=0
    One_target_built=0
    Two_targets_built=0
    Three_targets_built=0
    Four_targets_built=0
    Five_targets_built=0
    SixPlus_targets_built=0
    
    for coll in Collections:
        eight_cubes=[dictionary_for_collections[key] for key in coll]
        
        a_solution_list=[]
        for j in keysList_targets:
            a_solution_list.append(CountingSolutions(CornerCountVector(eight_cubes, dictionary_for_targets[j])))
    
        b=CountIfPositive(a_solution_list)
        
        if b==0:
            No_targets_built = No_targets_built+1
        
        if b==1:
            One_target_built = One_target_built+1
        
        if b==2:
            Two_targets_built = Two_targets_built+1
        
        if b==3:
            Three_targets_built = Three_targets_built+1
            
        if b==4:
            Four_targets_built = Four_targets_built+1
            
        if b==5: 
            Five_targets_built = Five_targets_built+1
            print(coll)
            print(a_solution_list)
            
        if b>5:
            SixPlus_targets_built = SixPlus_targets_built+1
            
    TargetBuild_distribution=[No_targets_built, One_target_built, Two_targets_built, Three_targets_built, Four_targets_built, Five_targets_built, SixPlus_targets_built]
    

    return TargetBuild_distribution


#NewCode, APR 9, 2024
#Input: dictionary_of_cubes, such as C36, C35, ...
#Output: a randomly selected collection of k cubes from the dictionary
## format of output, a list [[corner numbers cube 1], [corner numbers cube 2], ... , [corner numbers cube 8]]
def RandomKCubes(dictionary_of_cubes, k): 
    #a list of all dictionary keys, length of the list
    allkeysList = list(dictionary_of_cubes.keys())
    Number=len(allkeysList)
    
    #a random sample of k numbers from a range of 0-Number
    x=random.sample(range(Number), k)
    cube_names=[]
    collection=[]
    
    for i in x:
        cube_names.append(allkeysList[i])
    
    for j in cube_names:
        collection.append(dictionary_of_cubes[j])   
        
    return collection

#NEW CODE, Apr 9, 2024.
def NumberBuildable4aSetNINECubes(one_collection_of_nine, dictionary_for_targets): 
    keysList_targets = list(dictionary_for_targets.keys())
    NineSet = one_collection_of_nine
  
    Collections=list(it.combinations(NineSet, 8))
      
    current_solution_array=np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])     

    #for each subset of 8 cubes, determine how many cubes are buildable
    for coll in Collections:
        a_solution_list=[]
            
        for j in keysList_targets:
            a_solution_list.append(CountingSolutions(CornerCountVector(coll, dictionary_for_targets[j])))
            
        newarray = np.array([w for w in a_solution_list])
        current_solution_array = current_solution_array + newarray
  
    Number_buildable_cubes = np.count_nonzero(current_solution_array)
    
    return Number_buildable_cubes

#****

def DistributionBuildNum4SetsOfNine(dictionary_sets, number_samples):
    list_numberBuildable=[]
    for i in range(number_samples):
        nine_set = RandomKCubes(dictionary_sets, 9)
        n = NumberBuildable4aSetNINECubes(nine_set, dictionary_sets)
        list_numberBuildable.append(n)
        if n==8:
            print(nine_set)
        
    return list_numberBuildable






###########################################################################################
###########Code below used to display 6x6 grid of cubes in a set and display the cubes they can build
##########################################################################################
#uses import pandas as pd
#uses import matplotlib.pyplot as plt
def CubeCornersToNames(listOfcornernumbers, dictionary):
    list_of_names=[]
    for lis in listOfcornernumbers:
        a=find_key(dictionary, lis)
        list_of_names.append(a)   
        
    return list_of_names



def displayCubes(list_cube_names):
    blank_canvas = np.array([[90, 80, 80, 80, 80, 80], [80,90,80,80,80,80],[80,80,90,80,80,80], [80,80,80,90,80,80], [80,80,80,80,90,80], [80,80,80,80,80,90]])
    ArrayToAdd = np.zeros((6,6))

    cubenames = pd.DataFrame([['X', '126.453', '125.364', '123.645', '124.536', '135.426'], 
                          ['123.456', 'X', '136.524', '124.563', '126.345', '125.634'],
                          ['124.365', '134.526',       'X', '125.436', '123.654', '126.543'],
                          ['125.643', '123.564', '126.435',       'X', '134.625', '124.356'],
                          ['126.534', '125.346', '124.653', '135.624',       'X', '123.465'],
                          ['136.425', '124.635', '123.546', '126.354', '125.463', 'X']])

    for cube in list_cube_names:
        location = list(zip(*np.where(cubenames.values == cube)))
        #print(location)

        for entry in location:
            colID=entry[0] 
            rowID=entry[1] 
            #print(colID, rowID)
            
            ArrayToAdd[colID, rowID] = -50 
    ToDisplay = blank_canvas+ArrayToAdd
            
    
    # Define the color ranges and corresponding colors
    cmap = plt.cm.get_cmap('viridis', 10)
    bounds = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
    norm = plt.Normalize(min(bounds), max(bounds))
    colors = cmap(norm(bounds))

    # Create the colored grid
    fig, ax = plt.subplots(figsize=(4, 4))
    im = ax.imshow(ToDisplay, cmap=cmap, norm=norm)

    # Show the plot
    return plt.show()


def find_key(input_dict, target_value):
    for key, value in input_dict.items():
        if value == target_value:
            return key
    return None

def NumberBuildable4SetsSizeK(one_collection_of_k, dictionary_for_targets):
    k = len(one_collection_of_k)
    if k<8:
        return print("collection too small to build any targets")
    keysList_targets = list(dictionary_for_targets.keys())
    
    Collections=list(it.combinations(one_collection_of_k, 8))  
    current_solution_array=np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])     

    #for each subset of 8 cubes, determine how many cubes are buildable
    for coll in Collections:
        a_solution_list=[]
            
        for j in keysList_targets:
            a_solution_list.append(CountingSolutions(CornerCountVector(coll, dictionary_for_targets[j])))
            
        newarray = np.array([w for w in a_solution_list])
        current_solution_array = current_solution_array + newarray
  
    Number_buildable_cubes = np.count_nonzero(current_solution_array)
    
    return Number_buildable_cubes

def DistributionBuildNum4SetsOfK(k, dictionary_sets, number_samples):
    list_numberBuildable=[]
    for i in range(number_samples):
        set_size_k = RandomKCubes(dictionary_sets, k)
        n = NumberBuildable4SetsSizeK(set_size_k, dictionary_sets)
        list_numberBuildable.append(n)
        
    return list_numberBuildable



def DisplaySetCubesandBuildableTargets(set_of_cube_names, dictionary):
    #check the set has 8 or more cubes
    if len(set_of_cube_names)<8:
        return print("there are not enough cubes to build a target")
    Build=[]
    current_solution_list=[]
    newarray=[]
    #determine set of buildables for this set of cube names
    Keys4Targets=list(dictionary.keys())
    Collections=list(it.combinations(set_of_cube_names, 8))
    current_solution_array=np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
       
    for coll in Collections:
        eight_cubes=[dictionary[key] for key in coll]

        a_solution_list=[]
        for j in Keys4Targets: 
            a_solution_list.append(CountingSolutions(CornerCountVector(eight_cubes, dictionary[j])))
    
        
        newarray = np.array([w for w in a_solution_list])
        current_solution_array = current_solution_array + newarray
        current_solution_list = [num for num in current_solution_array]
        #Number_buildable_cubes = np.count_nonzero(current_solution_array)
    
    Build=current_solution_list
    
    Buildable=[]
    j=0
    for i in Build:
        if i > 0:
            Buildable.append(Keys4Targets[j])
        j=j+1
    
    numB = len(Buildable)
    print("these", numB, "cubes are Buildable:") 
          
    print(Buildable)
          
    print("the original set of cubes")
          
    displayCubes(set_of_cube_names)
          
    print("the set of buildable targets")
    
    return displayCubes(Buildable)
    
    